In [95]:
import os
import re
import glob
import datetime
import urllib.request
import pandas as pd

def clean_line(text):
    text = re.sub(r'<.*?>', '', text.decode('utf-8') if isinstance(text, bytes) else text).strip()
    text = re.sub(r'\s*,\s*', ',', text)
    return text

data_dir = "./vhi_data"
os.makedirs(data_dir, exist_ok=True)

base_url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean"

for province_id in range(1, 28):
    pattern = os.path.join(data_dir, f"vhi_id_{province_id}_*.csv")
    existing_files = glob.glob(pattern)
    latest_file = max(existing_files, key=os.path.getmtime) if existing_files else None

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    new_filename = os.path.join(data_dir, f"vhi_id_{province_id}_{timestamp}.csv")

    current_file = latest_file if latest_file else new_filename
    url = base_url.format(province_id)

    try:
        response = urllib.request.urlopen(url)
        raw_new_data = response.read().splitlines()
        
        clean_new_data = [clean_line(line) for line in raw_new_data if clean_line(line)]
        
        if latest_file:
            with open(latest_file, 'r', encoding='utf-8') as f:
                raw_old_data = f.readlines()

            clean_old_data = [clean_line(line) for line in raw_old_data if clean_line(line)]
            
            if len(clean_new_data) > 1 and len(clean_old_data) > 1:
                if sorted(set(clean_new_data[1:])) == sorted(set(clean_old_data[1:])):
                    print(f"No new data for provinceID {province_id}. File not updated.")
                else:
                    with open(new_filename, 'w', encoding='utf-8') as f:
                        f.writelines("\n".join(clean_new_data))
                    print(f"Updated data found for provinceID {province_id}. New file saved as {new_filename}.")
                    
                    os.remove(latest_file)
                    print(f"Old file {latest_file} deleted.")
                    
                    current_file = new_filename
            else:
                print(f"Error: Not enough data to compare for provinceID {province_id}.")
        else:
            with open(new_filename, 'w', encoding='utf-8') as f:
                f.writelines("\n".join(clean_new_data))
            print(f"File {new_filename} successfully downloaded.")
            
            current_file = new_filename

    except Exception as e:
        print(f"Error downloading provinceID {province_id}: {e}")

    print(f"Current working file for provinceID {province_id}: {current_file}")


File ./vhi_data\vhi_id_1_2025-03-23_01-21-53.csv successfully downloaded.
Current working file for provinceID 1: ./vhi_data\vhi_id_1_2025-03-23_01-21-53.csv
File ./vhi_data\vhi_id_2_2025-03-23_01-21-54.csv successfully downloaded.
Current working file for provinceID 2: ./vhi_data\vhi_id_2_2025-03-23_01-21-54.csv
File ./vhi_data\vhi_id_3_2025-03-23_01-21-55.csv successfully downloaded.
Current working file for provinceID 3: ./vhi_data\vhi_id_3_2025-03-23_01-21-55.csv
File ./vhi_data\vhi_id_4_2025-03-23_01-21-56.csv successfully downloaded.
Current working file for provinceID 4: ./vhi_data\vhi_id_4_2025-03-23_01-21-56.csv
File ./vhi_data\vhi_id_5_2025-03-23_01-21-57.csv successfully downloaded.
Current working file for provinceID 5: ./vhi_data\vhi_id_5_2025-03-23_01-21-57.csv
File ./vhi_data\vhi_id_6_2025-03-23_01-21-58.csv successfully downloaded.
Current working file for provinceID 6: ./vhi_data\vhi_id_6_2025-03-23_01-21-58.csv
File ./vhi_data\vhi_id_7_2025-03-23_01-21-59.csv successfu

In [96]:
id_mapping = {
    1: 25, 2: 27, 3: 26, 4: 18, 5: 3, 6: 4, 7: 8, 8: 22, 9: 23, 10: 24,
    11: 10, 12: 9, 13: 11, 14: 12, 15: 13, 16: 14, 17: 15, 18: 16, 19: 17,
    20: 19, 21: 20, 22: 21, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5
}

def process_files(folder):
    for filename in os.listdir(folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder, filename)

            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()
            
            header = lines[0]
            match = re.search(r"Province=\s*(\d+)", header)
            if match:
                province_id = int(match.group(1))
                new_id = id_mapping.get(province_id, province_id)
                header = re.sub(r"Province=\s*\d+", f"Province= {new_id}", header)
                lines[0] = header

                with open(file_path, "w", encoding="utf-8") as f:
                    f.writelines(lines)
            
            print(f"Updated Province in: {filename}")

process_files(data_dir)

Updated Province in: vhi_id_10_2025-03-23_01-22-01.csv
Updated Province in: vhi_id_11_2025-03-23_01-22-02.csv
Updated Province in: vhi_id_12_2025-03-23_01-22-03.csv
Updated Province in: vhi_id_13_2025-03-23_01-22-04.csv
Updated Province in: vhi_id_14_2025-03-23_01-22-05.csv
Updated Province in: vhi_id_15_2025-03-23_01-22-06.csv
Updated Province in: vhi_id_16_2025-03-23_01-22-06.csv
Updated Province in: vhi_id_17_2025-03-23_01-22-07.csv
Updated Province in: vhi_id_18_2025-03-23_01-22-08.csv
Updated Province in: vhi_id_19_2025-03-23_01-22-09.csv
Updated Province in: vhi_id_1_2025-03-23_01-21-53.csv
Updated Province in: vhi_id_20_2025-03-23_01-22-10.csv
Updated Province in: vhi_id_21_2025-03-23_01-22-11.csv
Updated Province in: vhi_id_22_2025-03-23_01-22-12.csv
Updated Province in: vhi_id_23_2025-03-23_01-22-13.csv
Updated Province in: vhi_id_24_2025-03-23_01-22-14.csv
Updated Province in: vhi_id_25_2025-03-23_01-22-15.csv
Updated Province in: vhi_id_26_2025-03-23_01-22-16.csv
Updated Pro

In [97]:
headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']

dfs = []

for file in os.listdir(data_dir):
    if file.endswith(".csv"):
        file_path = os.path.join(data_dir, file)
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                first_line = f.readline()
                match = re.search(r'Province=\s*(\d+)', first_line)
                if not match:
                    print(f"Province ID not found in file {file}, skipping...")
                    continue
                province_id = int(match.group(1))
            
            # Load data
            df = pd.read_csv(file_path, header=1, names=headers)
            df = df.drop(columns=["empty"], errors="ignore")
            df = df.set_index(["Year", "Week"])
            
            df["id"] = province_id
            dfs.append(df)
            print(f"File {file_path} processed successfully.")
            
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

if dfs:
    final_df = pd.concat(dfs)
    final_df = final_df.sort_values(by=["Year", "id"])
    final_df.to_csv("vhi_final.csv")
    print("File 'vhi_final.csv' saved successfully!")
    print(final_df.head())
else:
    print("No suitable files found for merging.")


File ./vhi_data\vhi_id_10_2025-03-23_01-22-01.csv processed successfully.
File ./vhi_data\vhi_id_11_2025-03-23_01-22-02.csv processed successfully.
File ./vhi_data\vhi_id_12_2025-03-23_01-22-03.csv processed successfully.
File ./vhi_data\vhi_id_13_2025-03-23_01-22-04.csv processed successfully.
File ./vhi_data\vhi_id_14_2025-03-23_01-22-05.csv processed successfully.
File ./vhi_data\vhi_id_15_2025-03-23_01-22-06.csv processed successfully.
File ./vhi_data\vhi_id_16_2025-03-23_01-22-06.csv processed successfully.
File ./vhi_data\vhi_id_17_2025-03-23_01-22-07.csv processed successfully.
File ./vhi_data\vhi_id_18_2025-03-23_01-22-08.csv processed successfully.
File ./vhi_data\vhi_id_19_2025-03-23_01-22-09.csv processed successfully.
File ./vhi_data\vhi_id_1_2025-03-23_01-21-53.csv processed successfully.
File ./vhi_data\vhi_id_20_2025-03-23_01-22-10.csv processed successfully.
File ./vhi_data\vhi_id_21_2025-03-23_01-22-11.csv processed successfully.
File ./vhi_data\vhi_id_22_2025-03-23_01

In [98]:
def load_data(file_path):
    return pd.read_csv(file_path, index_col=["Year", "Week"]).replace(-1, pd.NA).dropna()

def get_vhi_series(df, region_id, year):
    return df.query("id == @region_id and Year == @year")["VHI"]

final_df = load_data("vhi_final.csv")
print(get_vhi_series(final_df, 8, 2005))

Year  Week
2005  1       52.56
      2       53.32
      3       55.39
      4        56.2
      5       56.13
      6        56.5
      7       56.33
      8        56.9
      9       57.88
      10      58.26
      11       58.5
      12      58.78
      13       57.3
      14      53.32
      15      49.03
      16      47.26
      18      50.74
      19      52.84
      20      46.99
      21      54.93
      22      55.82
      23      57.42
      24       56.3
      25      55.71
      26      56.47
      27       56.3
      28      56.74
      29      61.33
      30      58.39
      31      60.51
      32      62.05
      33       61.9
      34      62.77
      35      61.78
      36      58.61
      37      55.98
      38      54.65
      39      53.14
      40      52.01
      41      51.73
      42      51.16
      43      50.91
      44      52.32
      45      54.23
      46      53.13
      47      49.57
      48      49.13
      49      48.69
      50      47.39
      51 

In [99]:
def get_extremes(df, region_ids, years):
    selected_data = df.loc[df.index.get_level_values("Year").isin(years) & df["id"].isin(region_ids), "VHI"]
    selected_data = selected_data.dropna()
    return {
        "min": float(selected_data.min()),
        "max": float(selected_data.max()),
        "mean": float(selected_data.mean()),
        "median": float(selected_data.median())
    }

print(get_extremes(final_df, [1, 4], range(1999, 2000)))

{'min': 21.56, 'max': 62.36, 'mean': 44.679423076923094, 'median': 45.269999999999996}


In [100]:
def get_vhi_for_range(df, region_ids, start_year, end_year):
    return df.loc[
        (df.index.get_level_values("Year") >= start_year) &
        (df.index.get_level_values("Year") <= end_year) &
        df["id"].isin(region_ids),
        ["id", "VHI"]
    ].set_index("id", append=True)["VHI"].unstack()

pd.set_option("display.max_rows", None)
print(get_vhi_for_range(final_df, [3, 5], 1995, 1996))

id             3      5
Year Week              
1995 4     57.56  49.83
     5     51.35  47.92
     6     42.64  45.16
     7     34.23  41.58
     8     29.13  39.64
     9     26.64  38.95
     10    25.49  38.95
     11    25.71   37.8
     12    26.72  41.42
     13    28.79  43.85
     14    29.97   45.1
     15    31.51  46.07
     16    33.09   46.2
     17    38.15  49.62
     18    43.17  50.76
     19    46.68  51.57
     20    49.03  53.89
     21    50.29  52.93
     22    49.21  50.15
     23    47.87  49.23
     24    49.91  49.43
     25    56.72  50.25
     26    61.35  49.28
     27    62.43  45.55
     28    62.02  44.65
     29    62.29  44.75
     30    62.96  45.42
     31    63.84  46.52
     32    64.83  47.43
     33    65.45  46.96
     34    65.59  44.99
     35    65.31  43.55
     36    63.61  46.98
     37    60.76  49.12
     38    60.09  51.85
     39     58.9  50.01
     40    57.49  47.33
     41    55.01  46.54
     42    52.34   48.3
     43    50.63

In [101]:
import pandas as pd

def detect_drought_years(df, threshold=20):
    df = df.reset_index()
    df = df[df["VHI"] != -1]
    
    total_regions = len(df["id"].unique())
    critical_regions = int((threshold / 100) * total_regions)
    drought_df = df[df["VHI"] < 15]
    drought_counts = drought_df.groupby("Year")["id"].nunique()
    drought_years = drought_counts[drought_counts >= critical_regions].index.tolist()
    extreme_droughts = drought_df[drought_df["Year"].isin(drought_years)].copy()
    
    regions = {
        1: "Вінницька", 2: "Волинська", 3: "Дніпропетровська", 4: "Донецька", 5: "Житомирська",
        6: "Закарпатська", 7: "Запорізька", 8: "Івано-Франківська", 9: "Київ", 10: "Київська",
        11: "Кіровоградська", 12: "Луганська", 13: "Львівська", 14: "Миколаївська", 15: "Одеська",
        16: "Полтавська", 17: "Рівненська", 18: "Республіка Крим", 19: "Севастополь", 20: "Сумська",
        21: "Тернопільська", 22: "Харківська", 23: "Херсонська", 24: "Хмельницька", 25: "Черкаська",
        26: "Чернівецька", 27: "Чернігівська"
    }
    extreme_droughts["Region"] = extreme_droughts["id"].map(regions)
    
    return extreme_droughts[["Year", "Week", "id", "Region", "VHI"]]

df = pd.read_csv("vhi_final.csv")

result = detect_drought_years(df, threshold=20)
print(result)

       Year  Week  id           Region    VHI
25316  2000    45   1        Вінницька  12.26
25317  2000    46   1        Вінницька  11.28
25318  2000    47   1        Вінницька  11.25
25319  2000    48   1        Вінницька  11.38
25320  2000    49   1        Вінницька  12.91
25321  2000    50   1        Вінницька  14.20
25729  2000    42   9             Київ  14.89
25730  2000    43   9             Київ  12.76
25731  2000    44   9             Київ   7.81
25732  2000    45   9             Київ   6.49
25733  2000    46   9             Київ   6.58
25734  2000    47   9             Київ   6.71
25735  2000    48   9             Київ   7.56
25736  2000    49   9             Київ   9.25
25737  2000    50   9             Київ  10.94
25738  2000    51   9             Київ  12.28
25783  2000    44  10         Київська  12.51
25784  2000    45  10         Київська  10.60
25785  2000    46  10         Київська  11.20
25786  2000    47  10         Київська  12.32
25787  2000    48  10         Київ